In [1]:
import pandas as pd
from pandarallel import pandarallel
from evaluate import load

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
df = pd.read_parquet('gs://scraped-news-article-data-null/mistral-sum-test-predicted.parquet')
df["predicted"] = df.predicted.str.strip()
df.head()

,body,published,question,summary,predicted
0,"Published: 2022-12-16\nPARIS, Dec 16 (Reuters)...",2022-12-16 15:39:00+00:00,"How are French, German, and Polish crops expec...",French crops predicted to withstand wintry wea...,"Wintry weather poses minimal risk to French, G..."
1,Published: 2023-10-25\n## In this article\nFol...,2023-10-25 20:12:41+00:00,What caused the surge in Endeavor stock?,Silver Lake's interest in taking Endeavor priv...,Endeavor stock surges as major shareholder Sil...
2,"Published: 2023-10-02\n""In mitigating climate ...",2023-10-02 11:54:30+00:00,What specific climate criticisms and accusatio...,Impossible to answer with given information,Impossible to answer with given information
3,Published: 2023-10-18\nA report published Tues...,2023-10-18 19:49:14+00:00,Which specific projects or companies will rece...,Impossible to answer with given information,Impossible to answer with given information
4,"Published: 2022-03-04\nMeanwhile, a new law th...",2022-03-04 18:46:53+00:00,What were the reasons cited by Russia for bloc...,Russia blocks Meta-owned Facebook amid invasio...,Russia blocks Meta-owned Facebook in the count...


In [3]:
from sklearn.metrics import classification_report

df["y"] = False
df["y_hat"] = False
df.loc[df.summary.str.contains("Impossible", case=False), "y"] = True
df["y"] = df.y.fillna(False)
df.loc[df.predicted.str.contains("Impossible", case=False), "y_hat"] = True
df["y_hat"] = df.y_hat.fillna(False)
print(classification_report(df.y, df.y_hat))

              precision    recall  f1-score   support

       False       0.95      0.94      0.94       985
        True       0.94      0.95      0.95      1093

    accuracy                           0.94      2078
   macro avg       0.94      0.94      0.94      2078
weighted avg       0.94      0.94      0.94      2078



In [4]:
df

,body,published,question,summary,predicted,y,y_hat
0,"Published: 2022-12-16\nPARIS, Dec 16 (Reuters)...",2022-12-16 15:39:00+00:00,"How are French, German, and Polish crops expec...",French crops predicted to withstand wintry wea...,"Wintry weather poses minimal risk to French, G...",False,False
1,Published: 2023-10-25\n## In this article\nFol...,2023-10-25 20:12:41+00:00,What caused the surge in Endeavor stock?,Silver Lake's interest in taking Endeavor priv...,Endeavor stock surges as major shareholder Sil...,False,False
2,"Published: 2023-10-02\n""In mitigating climate ...",2023-10-02 11:54:30+00:00,What specific climate criticisms and accusatio...,Impossible to answer with given information,Impossible to answer with given information,True,True
3,Published: 2023-10-18\nA report published Tues...,2023-10-18 19:49:14+00:00,Which specific projects or companies will rece...,Impossible to answer with given information,Impossible to answer with given information,True,True
4,"Published: 2022-03-04\nMeanwhile, a new law th...",2022-03-04 18:46:53+00:00,What were the reasons cited by Russia for bloc...,Russia blocks Meta-owned Facebook amid invasio...,Russia blocks Meta-owned Facebook in the count...,False,False
...,...,...,...,...,...,...,...
2073,"Published: 2022-07-03\nALMATY, July 3 (Reuters...",2017-09-01 12:35:39+00:00,What types of products are recommended to shop...,Impossible to answer with given information,Impossible to answer with given information,True,True
2074,"Published: 2022-11-21\nNEW YORK, Nov 21 (Reute...",2023-04-19 19:37:40+00:00,How will the defamation lawsuits filed against...,Impossible to answer with given information,Impossible to answer with given information,True,True
2075,Published: 2022-03-08\n## In this article\nFol...,2020-04-07 20:14:20+00:00,How many paying customers does Google's G Suit...,Impossible to answer with given information,Impossible to answer with given information,True,True
2076,Published: 2023-10-18\nAnalyst Jos Versteeg of...,2022-07-13 23:57:39+00:00,How are the wildfires in Portugal and Spain af...,Impossible to answer with given information,Impossible to answer with given information,True,True


In [5]:
import finetune.config as config
from finetune.common import create_summarization_metrics

_, rouge_metric, _ = create_summarization_metrics(None, None)
pure_df = df.loc[(~df.summary.str.contains("Impossible", case=False)) &
                 (~df.predicted.str.contains("Impossible", case=False))]
rouge_results = pure_df.parallel_apply(lambda row: rouge_metric([row["summary"]], [row["predicted"]]), axis=1)
rouge_results = pd.DataFrame(rouge_results.tolist())
rouge_results.head()

[nltk_data] Downloading package punkt to /home/sdai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sdai/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sdai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,rouge1,rouge2,rougeL,rougeLsum
0,46.3415,12.5000,26.8293,26.8293
1,59.3407,31.4607,50.5495,50.5495
2,44.6809,19.5652,31.9149,38.2979
3,45.0000,25.6410,32.5000,35.0000
4,50.4673,26.6667,41.1215,41.1215


In [6]:
rouge_results.to_excel('gs://scraped-news-article-data-null/mistral-sum-rouge.xlsx')

In [7]:
df = pd.read_parquet('gs://scraped-news-article-data-null/mistral-sum-3-test-predicted.parquet')
df["predicted"] = df.predicted.str.strip()
df.head()

,body,published,question,summary,predicted
0,"Published: 2022-12-16\nPARIS, Dec 16 (Reuters)...",2022-12-16 15:39:00+00:00,"How are French, German, and Polish crops expec...",French crops predicted to withstand wintry wea...,"Wintry weather in France, Germany, and Poland ..."
1,Published: 2023-10-25\n## In this article\nFol...,2023-10-25 20:12:41+00:00,What caused the surge in Endeavor stock?,Silver Lake's interest in taking Endeavor priv...,Endeavor stock surges after Silver Lake expres...
2,"Published: 2023-10-02\n""In mitigating climate ...",2023-10-02 11:54:30+00:00,What specific climate criticisms and accusatio...,Impossible to answer with given information,Impossible to answer with given information
3,Published: 2023-10-18\nA report published Tues...,2023-10-18 19:49:14+00:00,Which specific projects or companies will rece...,Impossible to answer with given information,Impossible to answer with given information
4,"Published: 2022-03-04\nMeanwhile, a new law th...",2022-03-04 18:46:53+00:00,What were the reasons cited by Russia for bloc...,Russia blocks Meta-owned Facebook amid invasio...,Russia blocks Meta-owned Facebook in the count...


In [8]:
df["y"] = False
df["y_hat"] = False
df.loc[df.summary.str.contains("Impossible", case=False), "y"] = True
df["y"] = df.y.fillna(False)
df.loc[df.predicted.str.contains("Impossible", case=False), "y_hat"] = True
df["y_hat"] = df.y_hat.fillna(False)
print(classification_report(df.y, df.y_hat))

              precision    recall  f1-score   support

       False       0.94      0.94      0.94       985
        True       0.94      0.95      0.95      1093

    accuracy                           0.94      2078
   macro avg       0.94      0.94      0.94      2078
weighted avg       0.94      0.94      0.94      2078



In [9]:
pure_df = df.loc[(~df.summary.str.contains("Impossible", case=False)) &
                 (~df.predicted.str.contains("Impossible", case=False))]
rouge_results = pure_df.parallel_apply(lambda row: rouge_metric([row["summary"]], [row["predicted"]]), axis=1)
rouge_results = pd.DataFrame(rouge_results.tolist())
rouge_results.head()

,rouge1,rouge2,rougeL,rougeLsum
0,28.9474,5.4054,18.4211,18.4211
1,49.5050,24.2424,27.7228,27.7228
2,57.7778,25.0000,35.5556,46.6667
3,43.8356,25.3521,27.3973,35.6164
4,39.5604,24.7191,32.9670,32.9670


In [10]:
rouge_results.to_excel('gs://scraped-news-article-data-null/mistral-sum-3-rouge.xlsx')

In [7]:
df = pd.read_parquet('gs://scraped-news-article-data-null/llama-chat-test-predicted.parquet')
df.loc[df.predicted.str.contains("IMPOSSIBLE", case=False), "predicted"] = "IMPOSSIBLE"
df.loc[df.predicted.str.contains("The context does not provide", case=False), "predicted"] = "IMPOSSIBLE"
df.head()

,body,published,question,summary,predicted
0,"Published: 2022-12-16\nPARIS, Dec 16 (Reuters)...",2022-12-16 15:39:00+00:00,"How are French, German, and Polish crops expec...",French crops predicted to withstand wintry wea...,"Based on the provided context, it is possibl..."
1,Published: 2023-10-25\n## In this article\nFol...,2023-10-25 20:12:41+00:00,What caused the surge in Endeavor stock?,Silver Lake's interest in taking Endeavor priv...,IMPOSSIBLE
2,"Published: 2023-10-02\n""In mitigating climate ...",2023-10-02 11:54:30+00:00,What specific climate criticisms and accusatio...,Impossible to answer with given information,"Based on the given context, the specific cli..."
3,Published: 2023-10-18\nA report published Tues...,2023-10-18 19:49:14+00:00,Which specific projects or companies will rece...,Impossible to answer with given information,IMPOSSIBLE
4,"Published: 2022-03-04\nMeanwhile, a new law th...",2022-03-04 18:46:53+00:00,What were the reasons cited by Russia for bloc...,Russia blocks Meta-owned Facebook amid invasio...,IMPOSSIBLE


In [8]:
df["y"] = False
df["y_hat"] = False
df.loc[df.summary.str.contains("IMPOSSIBLE", case=False), "y"] = True
df["y"] = df.y.fillna(False)
df.loc[df.predicted == "IMPOSSIBLE", "y_hat"] = True
df["y_hat"] = df.y_hat.fillna(False)
print(classification_report(df.y, df.y_hat))

              precision    recall  f1-score   support

       False       0.63      0.48      0.55       985
        True       0.62      0.75      0.68      1093

    accuracy                           0.62      2078
   macro avg       0.62      0.61      0.61      2078
weighted avg       0.62      0.62      0.61      2078



In [9]:
pure_df = df.loc[df.y == df.y_hat]
rouge_results = pure_df.parallel_apply(lambda row: rouge_metric([row["predicted"]], [row["summary"]]), axis=1)
rouge_results = pd.DataFrame(rouge_results.tolist())
rouge_results.head()

,rouge1,rouge2,rougeL,rougeLsum
0,21.2766,2.8777,12.7660,18.4397
1,40.0000,0.0000,40.0000,40.0000
2,40.0000,0.0000,40.0000,40.0000
3,25.4237,10.3448,15.2542,20.3390
4,40.0000,0.0000,40.0000,40.0000


In [10]:
rouge_results.to_excel('gs://scraped-news-article-data-null/llama-chat-rouge.xlsx')

In [11]:
df = pd.read_parquet('gs://scraped-news-article-data-null/orca-mistral-test-predicted.parquet')
df.loc[df.predicted.str.contains("IMPOSSIBLE", case=False), "predicted"] = "IMPOSSIBLE"
df.loc[df.predicted.str.contains("does not provide", case=False), "predicted"] = "IMPOSSIBLE"
df.head()

,body,published,question,summary,predicted
0,"Published: 2022-12-16\nPARIS, Dec 16 (Reuters)...",2022-12-16 15:39:00+00:00,"How are French, German, and Polish crops expec...",French crops predicted to withstand wintry wea...,French crops are expected to be unaffected by...
1,Published: 2023-10-25\n## In this article\nFol...,2023-10-25 20:12:41+00:00,What caused the surge in Endeavor stock?,Silver Lake's interest in taking Endeavor priv...,The surge in Endeavor stock was caused by the...
2,"Published: 2023-10-02\n""In mitigating climate ...",2023-10-02 11:54:30+00:00,What specific climate criticisms and accusatio...,Impossible to answer with given information,IMPOSSIBLE
3,Published: 2023-10-18\nA report published Tues...,2023-10-18 19:49:14+00:00,Which specific projects or companies will rece...,Impossible to answer with given information,The Biden Administration is investing $3.46 b...
4,"Published: 2022-03-04\nMeanwhile, a new law th...",2022-03-04 18:46:53+00:00,What were the reasons cited by Russia for bloc...,Russia blocks Meta-owned Facebook amid invasio...,Russia's media regulator blocked Meta-owned F...


In [12]:
df["y"] = False
df["y_hat"] = False
df.loc[df.summary.str.contains("IMPOSSIBLE", case=False), "y"] = True
df["y"] = df.y.fillna(False)
df.loc[df.predicted == "IMPOSSIBLE", "y_hat"] = True
df["y_hat"] = df.y_hat.fillna(False)
print(classification_report(df.y, df.y_hat))

              precision    recall  f1-score   support

       False       0.58      0.95      0.72       985
        True       0.90      0.39      0.55      1093

    accuracy                           0.66      2078
   macro avg       0.74      0.67      0.63      2078
weighted avg       0.75      0.66      0.63      2078



In [13]:
pure_df = df.loc[df.y == df.y_hat]
rouge_results = pure_df.parallel_apply(lambda row: rouge_metric([row["predicted"]], [row["summary"]]), axis=1)
rouge_results = pd.DataFrame(rouge_results.tolist())
rouge_results.head()

,rouge1,rouge2,rougeL,rougeLsum
0,28.5714,12.5000,20.4082,28.5714
1,34.2105,16.2162,23.6842,28.9474
2,40.0000,0.0000,40.0000,40.0000
3,37.6471,21.6867,28.2353,35.2941
4,43.1373,28.5714,35.2941,43.1373


In [14]:
rouge_results.to_excel('gs://scraped-news-article-data-null/orca-mistral-rouge.xlsx')